In [ ]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder



llm = ChatOpenAI(
    temperature=0.1,
)

loader = TextLoader("./document.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=200,
)
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful document information extractor. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        (
            "human",
            "{question}"
        ),
         MessagesPlaceholder(variable_name="history"),


    ]
)

memory = ConversationBufferMemory(
    llm = llm,
    return_messages= True,
    
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
        "history" : load_memory,
    }
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result)


In [3]:
invoke_chain("Is Aaronson guilty?")

content='Yes, according to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 2559, 'total_tokens': 2585, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-46f88706-e151-462e-81b1-569b35d2ded3-0' usage_metadata={'input_tokens': 2559, 'output_tokens': 26, 'total_tokens': 2585, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [4]:
invoke_chain("What message did he write in the table?")

content='The message he wrote on the table was: "FREEDOM IS SLAVERY" and then followed it with "TWO AND TWO MAKE FIVE" and "GOD IS POWER."' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 2612, 'total_tokens': 2653, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-fe8c6e17-8e56-4d26-84cd-a0baf2a506dc-0' usage_metadata={'input_tokens': 2612, 'output_tokens': 41, 'total_tokens': 2653, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [5]:
invoke_chain("Who is Julia?")

content='Julia is a character mentioned in the text who seems to have a significant connection to the main character. The text describes a moment where the main character cries out for Julia, expressing love and concern for her.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 2658, 'total_tokens': 2701, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4e868d7a-3fb9-4b10-b76c-e7d5e2299fc0-0' usage_metadata={'input_tokens': 2658, 'output_tokens': 43, 'total_tokens': 2701, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [8]:
invoke_chain("Who is Julia?")

content="I don't know." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 2703, 'total_tokens': 2709, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2560}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-8c7c8402-4890-45a3-84fd-f6eda4b4ed7d-0' usage_metadata={'input_tokens': 2703, 'output_tokens': 6, 'total_tokens': 2709, 'input_token_details': {'audio': 0, 'cache_read': 2560}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
